
## **Case Study 04 Machine Learning to predict public sentiment from text data.**

In [40]:
import numpy as np
import pandas as pd

In [41]:
data=pd.read_csv('/content/judge-1377884607_tweet_product_company.csv',encoding='ISO-8859-1')

In [42]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion


In [43]:
data=data[['tweet_text','is_there_an_emotion_directed_at_a_brand_or_product']]

In [44]:
data.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [45]:
data.is_there_an_emotion_directed_at_a_brand_or_product.value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [46]:
data[data['is_there_an_emotion_directed_at_a_brand_or_product']=='No emotion toward brand or product']='neutral'
data[data['is_there_an_emotion_directed_at_a_brand_or_product']=="I can't tell"]='noidea'
data[data['is_there_an_emotion_directed_at_a_brand_or_product']=='Positive emotion']='positive'
data[data['is_there_an_emotion_directed_at_a_brand_or_product']=='Negative emotion']='negative'

In [47]:
data.is_there_an_emotion_directed_at_a_brand_or_product.value_counts()

neutral     5389
positive    2978
negative     570
noidea       156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [48]:
pd.set_option('display.max_colwidth',None)
messages=data[['tweet_text','is_there_an_emotion_directed_at_a_brand_or_product']]
messages.columns=['text','label']
messages.head()

,text,label
0,negative,negative
1,positive,positive
2,positive,positive
3,negative,negative
4,positive,positive


In [49]:
X=messages['text']
y=messages['label'].values

In [50]:
y = y.reshape(-1,1)

In [51]:
#onehot encoding of target
from sklearn.preprocessing import OneHotEncoder
one=OneHotEncoder()
y = one.fit_transform(y)
y = y.toarray()

In [52]:
from keras.preprocessing import text
tokenizer = text.Tokenizer() 
tokenizer.fit_on_texts(list(messages['text']))
tokenized_texts = tokenizer.texts_to_sequences(messages['text'])

In [53]:
from keras.utils import pad_sequences
X=pad_sequences(tokenized_texts,maxlen=100)

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

## **RNN**

In [56]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SimpleRNN,Dropout

In [57]:
model=Sequential()

model.add(Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=128,input_length=100))
model.add(SimpleRNN(10))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='softmax'))

In [58]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [59]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          640       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 10)                1390      
                                                                 
 dense_2 (Dense)             (None, 50)                550       
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 4)                 204       
                                                                 
Total params: 2,784
Trainable params: 2,784
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.fit(X_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 7s 28ms/step - loss: 0.4475 - accuracy: 0.8558 - val_loss: 0.0428 - val_accuracy: 0.9808
Epoch 2/10
205/205 [==============================] - 6s 28ms/step - loss: 0.0385 - accuracy: 0.9956 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 3/10
205/205 [==============================] - 6s 27ms/step - loss: 0.0103 - accuracy: 0.9992 - val_loss: 7.4237e-04 - val_accuracy: 1.0000
Epoch 4/10
205/205 [==============================] - 5s 27ms/step - loss: 0.0062 - accuracy: 0.9992 - val_loss: 2.0821e-04 - val_accuracy: 1.0000
Epoch 5/10
205/205 [==============================] - 6s 27ms/step - loss: 0.0037 - accuracy: 0.9997 - val_loss: 7.4393e-05 - val_accuracy: 1.0000
Epoch 6/10
205/205 [==============================] - 6s 27ms/step - loss: 0.0030 - accuracy: 0.9995 - val_loss: 2.9233e-05 - val_accuracy: 1.0000
Epoch 7/10
205/205 [==============================] - 5s 26ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.3796e-05 - 

In [61]:
y_pred=model.predict(X_test)

57/57 [==============================] - 1s 8ms/step


In [62]:
#we have encoded the labels.

#so lets do the inverse one hot encoding for predicted labels
pred = []
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
    
#also inverse encoding for actual labels

test = []
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(test,pred)

1.0